## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,38.86,93,75,1.14,broken clouds
1,1,Albany,US,42.6001,-73.9662,85.64,41,100,3.00,overcast clouds
2,2,Zyryanovsk,KZ,49.7386,84.2719,45.39,74,15,2.21,few clouds
3,3,Bandarbeyla,SO,9.4942,50.8122,80.19,82,55,26.15,broken clouds
4,4,Cascais,PT,38.6979,-9.4215,64.62,54,20,5.99,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Albany,US,42.6001,-73.9662,85.64,41,100,3.00,overcast clouds
3,3,Bandarbeyla,SO,9.4942,50.8122,80.19,82,55,26.15,broken clouds
7,7,Sao Filipe,CV,14.8961,-24.4956,73.78,77,1,9.04,clear sky
9,9,Butaritari,KI,3.0707,172.7902,83.05,81,95,18.92,moderate rain
10,10,Lisakovsk,KZ,52.5369,62.4936,72.82,34,100,9.24,overcast clouds
12,12,Castro,BR,-24.7911,-50.0119,68.40,81,6,9.48,clear sky
13,13,Atuona,PF,-9.8000,-139.0333,80.42,67,47,15.39,scattered clouds
16,16,Puerto Ayora,EC,-0.7393,-90.3518,84.16,79,99,8.99,overcast clouds
18,18,Georgetown,MY,5.4112,100.3354,82.33,85,20,3.44,few clouds
19,19,Rikitea,PF,-23.1203,-134.9692,76.55,67,15,12.57,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,85.64,overcast clouds,42.6001,-73.9662,
3,Bandarbeyla,SO,80.19,broken clouds,9.4942,50.8122,
7,Sao Filipe,CV,73.78,clear sky,14.8961,-24.4956,
9,Butaritari,KI,83.05,moderate rain,3.0707,172.7902,
10,Lisakovsk,KZ,72.82,overcast clouds,52.5369,62.4936,
12,Castro,BR,68.40,clear sky,-24.7911,-50.0119,
13,Atuona,PF,80.42,scattered clouds,-9.8000,-139.0333,
16,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,
18,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
19,Rikitea,PF,76.55,few clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Not found... Skipping")
        

Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping
Not found... Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   388
Country                388
Max Temp               388
Current Description    388
Lat                    388
Lng                    388
Hotel Name             388
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))